In [1]:
import numpy as np
import pandas as pd

class MyGaussianNB:
    def __init__(self):
        self.sınıf_olasılıkları = {}  # P(y) sınıf öncelik olasılıkları
        self.ortalama = {}  # Her sınıf için ortalamalar
        self.standart_sapma = {}  # Her sınıf için standart sapmalar
        
    def fit(self, X, y):
        """ Modeli eğit: Her sınıf için ortalama, std ve olasılıkları hesapla. """
        sınıflar = np.unique(y)  # Benzersiz sınıflar (0 ve 1)
        for sınıf in sınıflar:
            X_sınıf = X[y == sınıf]  # Belirli sınıfa ait örnekleri al
            self.ortalama[sınıf] = X_sınıf.mean(axis=0)  # Özelliklerin ortalaması
            self.standart_sapma[sınıf] = X_sınıf.std(axis=0)  # Özelliklerin std sapması
            self.sınıf_olasılıkları[sınıf] = len(X_sınıf) / len(y)  # P(y) hesapla

    def _gaussian_olasılık(self, x, mean, std):
        """ Gauss dağılım fonksiyonu kullanarak olasılık hesapla. """
        if std == 0:
            std = 1e-6  # Bölme hatasını önlemek için küçük bir değer ekle
        exponent = np.exp(-((x - mean) ** 2 / (2 * (std ** 2))))
        return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

    def predict(self, X):
        """ Model ile tahmin yap: En yüksek olasılığa sahip sınıfı ata. """
        tahminler = []
        for x in X:
            olasılıklar = {}
            for sınıf in self.sınıf_olasılıkları:
                sınıf_olasilik = np.log(self.sınıf_olasılıkları[sınıf])  # P(y)
                for i in range(len(x)):  
                    sınıf_olasilik += np.log(self._gaussian_olasılık(x[i], 
                                                self.ortalama[sınıf][i], 
                                                self.standart_sapma[sınıf][i]))
                olasılıklar[sınıf] = sınıf_olasilik
            tahminler.append(max(olasılıklar, key=olasılıklar.get))  # En büyük olasılığa sahip sınıf
        return np.array(tahminler)

# Veri kümesini oku
df = pd.read_csv("data.csv")

# Veri hazırlama
X = df.drop(['liked'], axis=1).values  # Özellikler
y = df['liked'].values  # Hedef değişken

# Eğitim ve test veri seti
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli eğit
model = MyGaussianNB()
model.fit(X_train, y_train)

# Test verisinde tahmin yap
y_pred = model.predict(X_test)

# Model performansını değerlendirme
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(f"Model Doğruluk Oranı: {accuracy_score(y_test, y_pred):.4f}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Doğruluk Oranı: 0.8462

Confusion Matrix:
 [[13  6]
 [ 0 20]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.68      0.81        19
           1       0.77      1.00      0.87        20

    accuracy                           0.85        39
   macro avg       0.88      0.84      0.84        39
weighted avg       0.88      0.85      0.84        39



C:\Users\gorke\AppData\Local\Temp\ipykernel_25124\2608238651.py:34: RuntimeWarning: divide by zero encountered in log
  sınıf_olasilik += np.log(self._gaussian_olasılık(x[i],
